## Prediction model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
salary_df = pd.read_csv("dataset_for_prediction.csv")
#salary_df[(salary_df["Year"]==2000) & (salary_df["Tm"]=="SEA")]

In [3]:
teams = salary_df[["Tm", "Year", "Salaries"]]
teams = teams.groupby(["Tm", "Year"]).sum()
teams.rename(columns={'Salaries':'Teamcap'}, inplace=True)
dataset = pd.merge(left=salary_df, right=teams, left_on=["Tm", "Year"], right_on=["Tm", "Year"])
dataset['Wage%'] = round(dataset['Salaries'] / dataset['Teamcap']*100, 2)

In [4]:


data_dum = pd.get_dummies(columns=["Pos", "Tm"], data=dataset)
prediction_data = data_dum.drop(columns = ["Player"])
print(prediction_data)

       Unnamed: 0  Year   Age     G      MP   PER    TS%   3PAr    FTr  ORB%  \
0               0  2000  24.0  82.0  3070.0  20.6  0.570  0.288  0.282   3.2   
1               1  2000  23.0  27.0   361.0   4.3  0.310  0.147  0.042   1.6   
2              18  2000  30.0  81.0  2899.0  21.1  0.551  0.077  0.380   2.8   
3              61  2000  32.0  80.0  2129.0  12.7  0.550  0.004  0.563  12.8   
4             108  2000  27.0  81.0  2909.0  17.8  0.534  0.162  0.194   4.3   
5             124  2000  22.0  44.0   447.0  15.8  0.517  0.033  0.557  13.1   
6             133  2000  31.0  68.0  1488.0  15.6  0.539  0.000  0.303  14.0   
7               2  2000  29.0  82.0  2593.0  17.4  0.524  0.223  0.257   2.3   
8              78  2000  25.0  61.0   879.0   7.2  0.451  0.478  0.235   4.1   
9              98  2000  22.0  73.0  2583.0  19.8  0.550  0.255  0.409   3.5   
10            101  2000  24.0  79.0  1797.0  13.8  0.527  0.002  0.260  11.0   
11            126  2000  23.0  82.0  300

In [5]:
from sklearn.model_selection import train_test_split
X = prediction_data.copy().drop(columns=["Wage%", "Salaries"])
y = prediction_data["Wage%"]

X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.33, random_state=42)

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
standardized_data = scaler.fit_transform(X_train)
standardized_test = scaler.fit_transform(X_test)

### Predicting using Random Forest ###

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

# TODO
rf = RandomForestClassifier(n_estimators=100, max_depth=4,random_state=0).fit(X_train, y_train)
acc = accuracy_score(y_test, rf.predict(standardized_test))

ValueError: Unknown label type: 'continuous'

In [ ]:
acc

In [8]:
rayAllen = data_dum[data_dum["Player"] == "Kobe Bryant"]
rayAllen = rayAllen.drop(columns=["Player", "Salaries", "Wage%"])
#rayAllen_pred = rf.predict(rayAllen)
#print(rayAllen_pred)

### Predicting using Linear Regression ###

In [9]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(standardized_data, y_train)
print(lr.coef_)

[ 3.21870918e+00 -2.91186115e+00  2.08421909e-01 -6.89421356e-01
 -1.45236812e+00  7.61939798e-01  7.12314262e-02  1.22424600e-01
  8.71800359e-02  2.12192860e-01  7.10034023e-02 -2.76313441e-01
  1.97702435e-01  3.08732563e-02  4.62322933e-03 -2.30628134e-03
 -3.55239190e-01 -1.61266918e+00 -1.51939209e+00  3.38434315e+00
 -5.96793592e-01  3.27942253e-01  1.96391387e-01 -7.35643291e-01
  9.71450195e-01 -2.36238241e+11 -1.76045467e+11 -2.15977259e-01
  1.33250883e+11  5.94798607e+10  1.93641084e-02  3.45339934e+11
  1.41500619e+11  2.02189128e-01  8.23170490e-02  4.96520148e+10
  1.66784164e+00 -2.49805369e-02 -4.61533198e+10 -1.08285523e+11
  1.49270583e+11 -5.12870103e-01 -5.32679920e-01  1.68163834e-01
  7.97789849e-01 -8.97544929e-02 -2.25322265e+11 -4.81743804e+00
 -1.15465349e+00  1.07075879e+11  1.08485792e+11  1.05819767e+11
  1.01924894e+11  1.08871075e+11 -1.69671496e+10 -1.67157921e+10
 -1.08422726e+10 -1.18397746e+10 -5.20722313e+09 -1.50258297e+10
 -8.40608430e+09 -1.67157

In [10]:
print(lr.predict(rayAllen[rayAllen["Year"] == 2005]))
print(dataset[dataset["Player"] == "Kobe Bryant"]["Wage%"][dataset["Year"] == 2005])

[-4.03635665e+14]
2331    21.4
Name: Wage%, dtype: float64


### Predicting Key Players Salaries ###
#### Predicting new contract salary by year ####
Chosen players:
* Anthony Davis
* Damian Lillard
* Giannis Antetokounmpo

In [ ]:
players = ["Anthony Davis", "Damian Lillard", "Giannis Antetokounmpo"]
choosenModel = lr
for player in players:
    predictedSalary = choosenModel.predict(data_dum[(data_dum["Player"] == player) & (salary_df["Year"] == 2019 )].drop(columns=["Player", "Salaries"]))
    salaryNow = salary_df[(salary_df["Player"] == player) & (salary_df["Year"] == 2019 )]["Salaries"].iloc[0]
    print("Player: " + str(player)+"; "+ "Current Salary: "+ str(salaryNow) +"; "+"Predicted next season salary: "+ str(predictedSalary))
    

